# I03 `yield` và generator comprehension

## Mục đích

Giới thiệu chức năng của từ khóa `yield` để tạo ra generator và cách sử dụng generator comprehension.


## Generator

Hàm `map()` và `filter()` trong bài [I02](./02_lambda.ipynb) tạo ra các đối tượng không thực hiện tính toán ngay mà chờ đến khi người dùng sử dụng chúng vào một việc gì đó, ví dụ lặp qua chúng hoặc chuyển đổi chúng thành một dạng dữ liệu khác để lưu vào trong bộ nhớ. Generator chính là các đối tượng như thế. Để tạo ra một generator, chúng ta dùng từ khóa `yield` thay cho `return` khi khai báo một hàm.

Generator đặc biệt có lợi khi dữ liệu của chúng ta quá lớn, không thể tải vào bộ nhớ cùng một lúc. Lúc này, chúng ta giống như thể chỉ lưu trữ "cách tạo ra dữ liệu" vào trong bộ nhớ (thông qua generator), còn không thực thi vào tải dữ liệu thật sự vào bộ nhớ.

Chúng ta sẽ cùng xem một ví dụ.

In [1]:
def gen_first_10():
    i = 0

    while i < 10:
        yield i
        i += 1

gen_first_10()

<generator object gen_first_10 at 0x000002654FF93C10>

Như bạn thấy, hàm `gen_first_10()` trả về một đối tượng generator, không có con số nào ở đây được tạo ra cả. Nếu muốn lấy các dữ liệu trong generator này, chúng ta phải thực thể hóa nó, chẳng hạn, bằng `list()`.

In [2]:
list(gen_first_10())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Infinite generator (bộ sinh vô hạn)

Tuy nhiên, sẽ thật dại dột nếu generator của chúng ta tạo ra vô hạn phần tử như ví dụ dưới đây. Nếu không xác định điều kiện dừng vòng lặp và từ khóa `break`, vòng lặp `for` này sẽ chạy mãi mãi cho đến khi bạn kết thúc vòng lặp bằng bàn phím hoặc giá trị `i` quá lớn.

In [3]:
def infinite_generator():
    i = 0

    while True:
        yield i
        i += 1

for i in infinite_generator():
    if i > 100:
        break
    print(i, end=" ")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

Bạn có thể duyệt qua từng phần tử trong generator với hàm `next()`.

In [4]:
# Đây là một đối tượng generator
# không phải là toàn bộ danh sách giá trị tạo ra từ generator
inf_gen = infinite_generator()
print(next(inf_gen))
print(next(inf_gen))
print(next(inf_gen))

0
1
2


Các đối tượng có thể lặp được và có thể sử dụng hàm `next()` để duyệt qua phần tử tiếp theo trong đối tượng được gọi là **iterator** (bộ lặp), khác với iterable (đối tượng có thể lặp).

### Một ví dụ khác

Chúng ta sẽ cùng xem một ví dụ khác về infinite generator là công thức tính giá trị của số $e$ sử dụng khai triển Taylor.

$$e = \sum_{n=0}^{\infty} \frac{1}{n!}$$

Chungs ta có thể dừng việc in ra kết quả ước lượng của số $e$ khi sự khác biệt giữa hai kết quả ước lượng liên tiếp nhỏ hơn một giá trị `epsilon`.

In [5]:
def exp_x():
    # Khởi tạo giá trị lần đầu (số hạng đầu tiên của Taylor series)
    n = 0
    n_fact = 1
    e_n = 1

    while True:
        yield (n, e_n)

        n += 1               # n = n + 1 (đến số hạng tiếp theo của Taylor series)
        n_fact *= n          # n_fact = n_fact * n (để tính n!)
        e_n += 1 / n_fact    # Số hạng tiếp theo của Taylor series

e_gen = exp_x()

epsilon = 0.000001
prev_ex = 0
n, curr_ex = next(e_gen)     # Lấy ước lượng của e với 1 số hạng đầu tiên của Taylor series

while curr_ex - prev_ex > epsilon:     # Dừng vòng lặp khi chênh lệch giữa hai lần liên tiếp nhỏ hơn epsilon
    print(f"Iteration {n}: e = {curr_ex}")
    prev_ex = curr_ex            # Lưu kết quả lần trước
    n, curr_ex = next(e_gen)     # Lấy ước lượng tiếp theo

Iteration 0: e = 1
Iteration 1: e = 2.0
Iteration 2: e = 2.5
Iteration 3: e = 2.6666666666666665
Iteration 4: e = 2.708333333333333
Iteration 5: e = 2.7166666666666663
Iteration 6: e = 2.7180555555555554
Iteration 7: e = 2.7182539682539684
Iteration 8: e = 2.71827876984127
Iteration 9: e = 2.7182815255731922


## Generator comprehension

Bạn có thể tạo ra generator tương tự như cách chúng ta sử dụng list comprehension. Thay vì sử dụng cặp dấu `[]`, chúng ta sẽ dùng cặp dấu ngoặc tròn `()`.

In [6]:
gen_squares = (i ** 2 for i in range(20))
gen_squares

<generator object <genexpr> at 0x000002654F663740>

Cách làm này rất tiện lợi vì trong nhiều trường hợp bạn muốn tạo ra các chuỗi số trung gian để phục vụ tính toán nhưng không muốn lưu chúng vào list. Ví dụ công thức tính phương sai:

$$Var(X) = \frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2$$

Chúng ta sẽ phải tính bình phương của phần lỗi giữa x[i] và trung bình của các giá trị của x, nhưng việc lưu lại các giá trị này thực sự không cần thiết.

In [7]:
X = [1, 2, 3, 4, 5]
x_bar = sum(X) / len(X)

sum_of_squares = ((x - x_bar) ** 2 for x in X)
sum(sum_of_squares) / len(X)

2.0

## Lợi ích của generator

Lợi ích lớn nhất của generator là không bắt Python phải tải toàn bộ dữ liệu vào bộ nhớ, do đó thường được dùng trong trường hợp chúng ta phải xử lí dữ liệu lớn nhưng không thể chiếm nhiều không gian bộ nhớ.

Trong ví dụ này, file `bigfile.txt` có 10000 dòng (được tạo ra bằng đoạn mã dưới đây). Thật ra chúng ta có thể hoàn toàn đọc cả file này vào bộ nhớ, nhưng tại sao phải làm vậy nếu chỉ muốn lấy ra một vài dòng?

```python
import hashlib
with open("bigfile.txt", "w") as f:
    for i in range(10000):
        f.write("{}: {}\n".format(i, md5(str(i).encode()).hexdigest()))
```

Chúng ta sẽ tạo một generator để đọc từng dòng trong file này và chỉ giữ lại trong bộ nhớ danh sách những dòng bắt đầu bằng `"853"`. File nói trên được mở bằng lệnh `open("bigfile.txt", "r")`, lưu vào trong một biến `f`, cung cấp biến `f` này cho hàm tạo generator, và đọc từng dòng bằng lệnh `f.readline()`. Sau khi đã đọc, chúng ta đóng file bằng lệnh `f.close()`.

In [8]:
def read_bigfile(f):
    while True:
        line = f.readline()
        if not line:    # Nếu kết thúc file thì dừng lại không đọc nữa
            break
        yield line

f = open("bigfile.txt", "r")
lines = [line for line in read_bigfile(f) if line.startswith("853")]
f.close()

lines

['853: aff1621254f7c1be92f64550478c56e6\n',
 '8530: fa385fb5a8e79b41a0d37b12c9f95996\n',
 '8531: 749a8e6c231831ef7756db230b4359c8\n',
 '8532: a383d162a97be62a400e00c320eae9c7\n',
 '8533: 1763ea5a7e72dd7ee64073c2dda7a7a8\n',
 '8534: 92b78b60f9d00a0ac34898be97d15188\n',
 '8535: db53e24fbc4e5a62aaa6e92f6bd1167f\n',
 '8536: 2c2dc47463ad7f389023f2b0fe1058cf\n',
 '8537: 5c843bd82838f70b8321b95e2f1a84ef\n',
 '8538: a378383b89e6719e15cd1aa45478627c\n',
 '8539: 618790ae971abb5610b16c826fb72d01\n']

---

[Bài trước](./02_lambda.ipynb) - [Danh sách bài](../README.md) - [Bài sau](./04_bit.ipynb)